# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [73]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [22]:
data.head(10)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
5,b,1,4,2,6,1,0,0,0,266,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
6,b,1,4,2,5,0,1,0,0,13,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
7,b,1,3,4,21,0,1,0,1,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
8,b,1,4,3,3,0,0,0,0,316,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
9,b,1,4,2,6,0,1,0,0,263,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private


In [71]:
print(data.shape)

(4870, 65)


#### 1. What test is appropriate for this problem? Does CLT apply?

- We want to determine if race has any significant impat on the rate of callbacks. The data must be divided into two groups based on race column, and we want to determine if the rate of callback is different between these two populations.


In [30]:
data.race.value_counts()
# number of callbacks for black-sounding names
print ('Number of total callbacks :', sum(data.call))
print ('Number of callbacks for black-sounding names:', sum(data[data.race=='b'].call))
print ('Number of callbacks for white-sounding names:', sum(data[data.race=='w'].call))


Number of total callbacks : 392.0
Number of callbacks for black-sounding names: 157.0
Number of callbacks for white-sounding names: 235.0


- This is a comparison between the proportions of sample, and we will use a two-sample t test here.
- According to the description for the collected data, the race values are assigned randomly to each resume, therefore we may consider the samples are independent.
- The sample size of each group (divided based on race) is above 30, we can also see from above, number of samples that received and not received callback both exceed 30, hence we are safe to consider that CLT holds.


#### 2. What are the null and alternate hypotheses?

- Let $p_w$ denote the proportion of resume callback for white-sounding name, and $p_b$ be the proportion of resume callback for black-sounding name. We consider the null hypothesis and the alternative hypothesis as

    $H_0: p_w = p_b$

    $H_1: p_w \neq p_b$

    As a standard, we assume that we test the hypothesis with the significance level $\alpha = 0.05$

- Null hypothesis shows that there is no role of race in resume callback and total callbacks for black-sounding names and white-sounding names are same.

- Alternative hypothesis represent that total callbacks for black-sounding names and white-sounding names are same and that there is no role of race in resume callback.


#### 3. Compute margin of error, confidence interval, and p-value.

- lets first calculate mean and the difference of mean ( 2 possible ways)
- after that we will try to calculate the t statistics and p value of means.


In [46]:
data_racew = data[data.race == 'w']
data_raceb = data[data.race == 'b']
n_w = len(data_racew)
p_w = sum(data_racew.call) / n_w
n_b = len(data_raceb)
p_b = sum(data_raceb.call) / n_b


print('The sample mean for \'b\' race callbacks is {:.2f}.'.format(data_raceb['call'].mean()))
print('The sample mean for \'w\' race callbacks is {:.2f}.'.format(data_racew['call'].mean()))
print('The sample mean difference is {:.3f}'.format(data_racew['call'].mean() - data_raceb['call'].mean()))
print('The difference in proportion is ', p_w - p_b)


The sample mean for 'b' race callbacks is 0.06.
The sample mean for 'w' race callbacks is 0.10.
The sample mean difference is 0.032
The difference in proportion is  0.0320328542094


- we first need to calculate the p of comined sum

     $\bar{p} = \frac{n_w p_w + n_b p_b}{n_w+n_b}$
 
 
 
- We need to calculate the sigma of differences

    $\sigma_{p_w-p_b} = \sqrt{\frac{\bar{p}(1-\bar{p})}{n_w} + \frac{\bar{p}(1-\bar{p})}{n_b}}$




In [70]:

p_all = (sum(data_racew.call) + sum(data_raceb.call)) / (n_w + n_b)
sigma = ((p_all*(1-p_all) / n_w) + (p_all*(1-p_all) / n_b)) ** 0.5


confidence = 0.95
confidence_z = 1 - (1-confidence)/2
ME = sigma * stats.norm.ppf(confidence_z)
CI = (-ME, ME)
print('The 95% confidence interval for the difference in proportion is ', CI)
print('Margin of error is ', (ME - (-ME))/2)


The 95% confidence interval for the difference in proportion is  (-0.015281631502169232, 0.015281631502169232)
Margin of error is  0.0152816315022


In [62]:
mean_racew = data[data.race == 'w'].call.mean()
var_racew = data[data.race == 'w'].call.var()
mean_raceb = data[data.race == 'b'].call.mean()
var_raceb = data[data.race == 'b'].call.var()

SE = np.sqrt(var_racew/n_w + var_raceb/n_b)

print('Mean of white-sounding names callback is {:.2f}'.format(mean_racew))
print('Mean of black-sounding names callback is {:.2f}'.format(mean_raceb))
print('Standard error is: ', SE)

two_sample = stats.ttest_ind(data_raceb['call'], data_racew['call'])
print('The t statistic is %.3f and the p-value is %.6f.' % two_sample)


Mean of white-sounding names callback is 0.10
Mean of black-sounding names callback is 0.06
Standard error is:  0.00778490682688
The t statistic is -4.115 and the p-value is 0.000039.


#### 4. Write a story describing the statistical significance in the context or the original problem.

The calculated p value is very less. This means that we can reject the null hypothesis in favor of the alternate hypothesis that the number of interview callbacks do depend on whther the candidate's name is black-sounding or white-sounding. In fact, the difference between the white-sounding resume callback proportion and black-sounding resume callback proportion is statistically significant. 

Given the p-value, test statistic, and 95% confidence interval, there is a statitistically significant difference between the number of calls a black sounding name receives and that of a white sounding name. We are 95% confident that white sounding names receive more between 0.047 and 0.017 more calls than black sounding names. This result, however, is not practically significant and it seems as though the true difference in calls is not completely explained by race alone.



#### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis indicates that race/name is somewhat is a factor in callback success, but it is not a large enough factor such that it can completely describes why certain type of candidates receive more callbacks than others.

There may be some other features in the dataset such as education, years of experience, and previous jobs just to name a few which might be the significant reason behind it. It is possible that the callback number would depend on these additional factors as well. Therefore, we cannot conclude that the race is the most (and/or only) important factor. 

In order to find the most important factor, we will have to explore the differences in the others features and each factor needs to be studied separately and see its effect on the number of callbacks. In other words, we would need to find the effect size for all factors/columns. 